# Overview
This notebook explores the cleaned ramen-ratings CSV. Any engineered features or scripts will be added to explore.py.

# Findings
1. Ramen packaging and five-star ratings are independent.
2. Ramen country of origin and five-star ratings **have a dependent relationship.**

In [1]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle

In [2]:
# create train split for exploration
train, _, _ = wrangle.prep_explore()
print('')

# check work
train.info()

Train size: (1515, 5) Validate size: (506, 5) Test size: (506, 5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 2566 to 2127
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   brand       1515 non-null   object
 1   product     1515 non-null   object
 2   package     1515 non-null   object
 3   country     1515 non-null   object
 4   five_stars  1515 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 60.7+ KB


# Exploring Ramen Packaging
Before going into it, let's check if there's a dependent relationship between packaging and our target.

Hypotheses:
- $H_0$: Packaging and five-star ratings are independent.
- $H_a$: Packaging and five-star ratings have a dependent relationship.

Confidence interval: 95%

In [3]:
# set confidence interval
alpha = .05

In [4]:
# check dependence of packaging and target
package_5star_crosstab = pd.crosstab(train.package, train.five_stars)
_, p, _, _ = stats.chi2_contingency(package_5star_crosstab)

In [5]:
# check if p is significant
if p < alpha:
    print("Packaging and five-star ratings have a dependent relationship with 95% confidence.")
    print("p-value:", p)
else:
    print("Packaging and five-star ratings are independent, did not pass 95% confidence interval.")
    print("p-value:", p)

Packaging and five-star ratings are independent, did not pass 95% confidence interval.
p-value: 0.43924606238117814


**Packaging and five-star ratings are independent.** We will not use 'package' in our predictive model.

# Exploring Ramen Country of Origin
Before going into it, let's check if there's a dependent relationship between country and our target.

Hypotheses:
- $H_0$: Country of origin and five-star ratings are independent.
- $H_a$: Country of origin and five-star ratings have a dependent relationship.

Confidence interval: 95%

In [6]:
# set confidence interval
alpha = .05

In [7]:
# create crosstab for test
country_5star_crosstab = pd.crosstab(train.country, train.five_stars)
# limit only to countries with sufficient value counts in crosstab (an assumption of chi-square)
enough_values_mask = (country_5star_crosstab[False] > 5) & (country_5star_crosstab[True] > 5)
# run chi-square test
_, p, _, _ = stats.chi2_contingency(country_5star_crosstab[enough_values_mask])

In [8]:
# check if p is significant
if p < alpha:
    print("Country of origin and five-star ratings have a dependent relationship with 95% confidence.")
    print("p-value:", p)
else:
    print("Country of origin and five-star ratings are independent, did not pass 95% confidence interval.")
    print("p-value:", p)

Country of origin and five-star ratings have a dependent relationship with 95% confidence.
p-value: 1.680501270556502e-06


**Country of origin and five-star ratings have a dependent relationship.** We will further explore country of origin and consider using it in our model.

In [9]:
train.groupby('country').five_stars.count()

country
Australia       11
Bangladesh       3
Brazil           2
Cambodia         5
Canada          26
China          112
Colombia         3
Germany         15
Hong Kong       88
Hungary          3
India           21
Indonesia       71
Japan          213
Malaysia        92
Mexico          17
Myanmar          8
Nepal            9
Netherlands      8
Pakistan         5
Philippines     27
Singapore       57
South Korea    188
Taiwan         132
Thailand       108
UK              34
USA            193
Vietnam         64
Name: five_stars, dtype: int64